Progetto Data Mining A.A. 2019/2020
=====

Asteroidi e comete: analisi del diametro, dell'orbita e della pericolosità
---

Francesco Pasceri matricola 204963

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

print('imported')

imported


Fase 2: Pre processing dei dati per le analisi di regressione, classificazione e anomaly detection
---

## 1. Import data


<b><u>
Il punto di partenza è ancora una volta il caricamento, tramite la libreria pandas, del nostro dataset.
</u></b>

In [2]:
data_org = pd.read_csv('jpl_db.csv', low_memory=False)
data_org.head()

,full_name,neo,pha,H,G,M1,diameter,albedo,rot_per,GM,...,ad,n,per_y,moid,moid_jup,t_jup,class,first_obs,n_obs_used,condition_code
0,1 Ceres (A801 AA),N,N,3.40,0.12,NaN,939.400,0.0900,9.074170,62.6284,...,2.979647,0.213885,4.608202,1.59478,2.09753,3.309,MBA,1995-01-05,1030.0,0
1,2 Pallas (A802 FA),N,N,4.20,0.11,NaN,545.000,0.1010,7.813200,14.3000,...,3.411748,0.213345,4.619880,1.23429,1.85093,3.042,MBA,1804-08-27,8477.0,0
2,3 Juno (A804 RA),N,N,5.33,0.32,NaN,246.596,0.2140,7.210000,NaN,...,3.353865,0.226129,4.358696,1.03429,2.18899,3.299,MBA,1804-10-17,7188.0,0
3,4 Vesta (A807 FA),N,N,3.00,0.32,NaN,525.400,0.4228,5.342128,17.8000,...,2.570926,0.271609,3.628837,1.13948,2.46988,3.535,MBA,1950-09-23,9397.0,0
4,5 Astraea (A845 XA),N,N,6.90,NaN,NaN,106.699,0.2740,16.806000,NaN,...,3.065455,0.238661,4.129814,1.09575,1.95968,3.396,MBA,1845-12-15,3034.0,0


In [3]:
data = data_org.copy()

<b><u>
Il primo step di pre-processing è quello di gestire le mancanze di valori fisici importanti, quali diametro e albedo. La risoluzione si basa sulla semplice <font color='red'>rimozione dei record</font>.
    
Il processo coinvolge anche l'attributo pha che è rilevante per le analisi successive.
</u></b>

In [4]:
#remove meaningless rows
data = data[data.diameter.notnull()]
data = data[data.albedo.notnull()]
data = data[data.pha.notnull()]

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139014 entries, 0 to 977556
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   full_name       139014 non-null  object 
 1   neo             139014 non-null  object 
 2   pha             139014 non-null  object 
 3   H               135434 non-null  float64
 4   G               118 non-null     float64
 5   M1              0 non-null       float64
 6   diameter        139014 non-null  float64
 7   albedo          139014 non-null  float64
 8   rot_per         13577 non-null   float64
 9   GM              13 non-null      float64
 10  e               139014 non-null  float64
 11  a               139014 non-null  float64
 12  q               139014 non-null  float64
 13  i               139014 non-null  float64
 14  om              139014 non-null  float64
 15  w               139014 non-null  float64
 16  ma              139014 non-null  float64
 17  ad        

<b><u>
Il secondo step è quindi gestire i valori mancanti dell'attributo H (magnitudo) fornendo il valore della <font color='red'>moda</font>, ovvero quello più frequente.    
</u></b>

In [5]:
#filling
value = data.H.mode()[0]
data['H'].fillna(value, inplace=True)

<b><u>
Ancora, osservando i dati alcuni sono stringhe, altri binari (Y/N), altri ancora sono delle date. Per valutarli correttamente, dapprima mappiamo i binari in attributi discreti ma ancora binari: essendo la classe YES quella più rara la indicheremo con 1, il NO diventerà 0.
</u></b>

In [6]:
#converting into numerical
data['moid'] = pd.to_numeric(data['moid'])
data['pha'] = pd.to_numeric(data['pha'].map(dict(Y=1, N=0)))
data['neo'] = pd.to_numeric(data['neo'].map(dict(Y=1, N=0)))
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139014 entries, 0 to 977556
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   full_name       139014 non-null  object 
 1   neo             139014 non-null  int64  
 2   pha             139014 non-null  int64  
 3   H               139014 non-null  float64
 4   G               118 non-null     float64
 5   M1              0 non-null       float64
 6   diameter        139014 non-null  float64
 7   albedo          139014 non-null  float64
 8   rot_per         13577 non-null   float64
 9   GM              13 non-null      float64
 10  e               139014 non-null  float64
 11  a               139014 non-null  float64
 12  q               139014 non-null  float64
 13  i               139014 non-null  float64
 14  om              139014 non-null  float64
 15  w               139014 non-null  float64
 16  ma              139014 non-null  float64
 17  ad        

<b><u>
Successivamente, rimuoviamo gli attributi, quindi intere colonne, che non hanno sufficienti valori diversi da <i>NULL</i>. Inoltre, procediamo con la rimozione delle colonne delle date, dei nomi dei record e del numero di osservazioni che non inducono alcuna informazione rilevante per le analisi.
</u></b>

In [7]:
#removing columns
vect = np.asarray(pd.DataFrame(data.isnull().sum() / data.shape[0]).T).flatten()
colstoRem = np.asarray(np.where(vect>0.5)).flatten().tolist()
print('index of columns to remove = ', colstoRem)
data.drop(data.columns[colstoRem], axis=1, inplace=True)
data.drop(['first_obs', 'n_obs_used', 'full_name'], axis=1, inplace=True)
data.info()

index of columns to remove =  [4, 5, 8, 9]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 139014 entries, 0 to 977556
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   neo             139014 non-null  int64  
 1   pha             139014 non-null  int64  
 2   H               139014 non-null  float64
 3   diameter        139014 non-null  float64
 4   albedo          139014 non-null  float64
 5   e               139014 non-null  float64
 6   a               139014 non-null  float64
 7   q               139014 non-null  float64
 8   i               139014 non-null  float64
 9   om              139014 non-null  float64
 10  w               139014 non-null  float64
 11  ma              139014 non-null  float64
 12  ad              139014 non-null  float64
 13  n               139014 non-null  float64
 14  per_y           139014 non-null  float64
 15  moid            139014 non-null  float64
 16  moid_jup     

<b><u>
Lo step finale è quello di sostituire le stringhe e le sigle con parametri numeri definiti dai dizionari.
<br />
Convertiamo in numero anche il condition code, o <i>Uncertainty Parameter</i>
</u></b>

In [8]:
classes = data['class'].unique()
codes = np.arange(classes.shape[0])
dict_classes = {}
for key,val in zip(classes, codes):
    dict_classes[key] = val

data['class'] = pd.to_numeric(data['class'].map(dict_classes))
data['condition_code'] = pd.to_numeric(data['condition_code'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139014 entries, 0 to 977556
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   neo             139014 non-null  int64  
 1   pha             139014 non-null  int64  
 2   H               139014 non-null  float64
 3   diameter        139014 non-null  float64
 4   albedo          139014 non-null  float64
 5   e               139014 non-null  float64
 6   a               139014 non-null  float64
 7   q               139014 non-null  float64
 8   i               139014 non-null  float64
 9   om              139014 non-null  float64
 10  w               139014 non-null  float64
 11  ma              139014 non-null  float64
 12  ad              139014 non-null  float64
 13  n               139014 non-null  float64
 14  per_y           139014 non-null  float64
 15  moid            139014 non-null  float64
 16  moid_jup        139014 non-null  float64
 17  t_jup     

<b><u>
Ultimo step è l'inserimento delle grandezze logaritmiche su alcuni parametri fisici definiti.
</b></u>

In [9]:
for column in ['albedo','n','H','ad','e','om','i','q']:
    data['log('+column+')'] = data[column].apply(np.log)
data.head()

,neo,pha,H,diameter,albedo,e,a,q,i,om,...,class,condition_code,log(albedo),log(n),log(H),log(ad),log(e),log(om),log(i),log(q)
0,0,0,3.40,939.400,0.0900,0.076009,2.769165,2.558684,10.594067,80.305531,...,0,0,-2.407946,-1.542316,1.223775,1.091805,-2.576903,4.385838,2.360294,0.939493
1,0,0,4.20,545.000,0.1010,0.229972,2.773841,2.135935,34.832932,173.024741,...,0,0,-2.292635,-1.544847,1.435085,1.227225,-1.469797,5.153435,3.550563,0.758904
2,0,0,5.33,246.596,0.2140,0.256936,2.668285,1.982706,12.991043,169.851482,...,0,0,-1.541779,-1.486651,1.673351,1.210113,-1.358927,5.134924,2.564260,0.684462
3,0,0,3.00,525.400,0.4228,0.088721,2.361418,2.151909,7.141771,103.810804,...,0,0,-0.860856,-1.303390,1.098612,0.944266,-2.422253,4.642570,1.965961,0.766356
4,0,0,6.90,106.699,0.2740,0.190913,2.574037,2.082619,5.367427,141.571026,...,0,0,-1.294627,-1.432710,1.931521,1.120196,-1.655936,4.952802,1.680349,0.733626


In [10]:
import pickle

with open('data','wb') as f:
    pickle.dump(data,f)